### City Traffic Index

Our **City Traffic Index** measures the overall movement intensity across Warsaw’s districts.
It’s based on **anonymized network data**, showing how many unique users connect to different cells during **morning**, **noon**, and **evening** hours.
By aggregating these observations, we capture how actively people move through each area — higher scores mean heavier traffic and more dynamic flow of people.
This provides a **data-driven view** of daily city mobility patterns in an accessible, comparable way.

In [13]:
import pandas as pd

In [3]:
df = pd.read_csv("./data/hackplay_warszawa_with_districts.csv")

In [4]:
df["start_dttm"] = pd.to_datetime(df["start_dttm"], errors="coerce")
tmp = df.dropna(subset=["start_dttm"]).copy()
hours = tmp["start_dttm"].dt.hour

In [5]:
def time_bucket(h: int) -> str:
    if 5 <= h < 11:
        return "morning"
    if 11 <= h < 17:
        return "noon"
    if 17 <= h < 23:
        return "evening"
    return "night"

In [6]:
tmp["time_bucket"] = hours.map(time_bucket)
tmp = tmp[tmp["time_bucket"].isin(["morning", "noon", "evening"])]

In [7]:
agg = (
    tmp.groupby(["district", "time_bucket"], as_index=False)["user_id"]
       .nunique()
       .rename(columns={"user_id": "unique_users"})
)

In [8]:
def scale_0_100(x: pd.Series) -> pd.Series:
    x_min, x_max = x.min(), x.max()
    if x_max == x_min:
        return pd.Series(100.0, index=x.index)
    return (x - x_min) / (x_max - x_min) * 100.0

agg["score_0_100"] = (
    agg.groupby("time_bucket")["unique_users"]
       .transform(scale_0_100)
       .round(1)
)

In [11]:
result = agg.sort_values(["time_bucket", "score_0_100"], ascending=[True, False]).reset_index(drop=True)
result.to_csv("./data/warsaw_district_mobility_scores.csv", index=False)

result.head(200)

,district,time_bucket,unique_users,score_0_100
0,śródmieście,evening,32,100.0
1,wola,evening,19,58.1
2,praga południe,evening,18,54.8
3,mokotów,evening,14,41.9
4,białołęka,evening,13,38.7
5,ursynów,evening,13,38.7
6,bemowo,evening,10,29.0
7,bielany,evening,10,29.0
8,targówek,evening,10,29.0
9,wawer,evening,9,25.8
